In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from pymor.basic import *
from problems import whole_problem
from discretizer import discretize_instationary_IP
from pymor.parameters.base import ParameterSpace

from model import InstationaryModelIP

In [6]:
N = 10                                                                         # FE Dofs = (N+1)^2                                                
noise_level = 1e-5        
nt = 10
fine_N = 2 * N

dims = {
    'N': N,
    'nt': nt,
    'fine_N': fine_N,
    'state_dim': (N+1)**2,
    'fine_state_dim': (fine_N+1)**2,
    'diameter': np.sqrt(2)/N,
    'fine_diameter': np.sqrt(2)/fine_N,
    'par_dim': (N+1)**2,
    'output_dim': 1,                                                                                                                                                                         # options to preassemble affine components or not
} 

bounds = [0.001*np.ones((dims['par_dim'],)), 10e2*np.ones((dims['par_dim'],))]

model_parameter = {
    'T_final' : 1,
    'noise_percentage' : None,
    'noise_level' : 0.05,
    'q_circ' : 3*np.ones((nt, dims['par_dim'])), 
    'q_exact' : None,
    'bounds' : bounds,
    #'parameter_space' : ParameterSpace(analytical_problem.parameters, bounds) 
    'parameters' : None
}

In [7]:
print('Construct problem..')                                                     
analytical_problem, q_exact, N, problem_type, exact_analytical_problem, energy_problem = whole_problem(
                                                        N = N,
                                                        parameter_location = 'reaction',
                                                        boundary_conditions = 'dirichlet',
                                                        exact_parameter = 'Kirchner',
                                                       )


model_parameter['q_exact'] = q_exact
model_parameter['parameters'] = analytical_problem.parameters

Construct problem..


In [8]:
type(analytical_problem.parameters)

pymor.parameters.base.Parameters

In [9]:
print('Discretizing problem...')                                                
# discretize analytical problem to obtain inverse problem fom
building_blocks = discretize_instationary_IP(analytical_problem,
                            model_parameter,
                            dims, 
                            problem_type
                        ) 

model = InstationaryModelIP(
    *building_blocks,
    dims = dims,
    model_parameter = model_parameter
)

Discretizing problem...


Accordion(children=(HTML(value='', layout=Layout(height='16em', width='100%')),), titles=('Log Output',))

noise percentage is 0.09589269187583464
noise_level is 0.05


In [10]:
q_exact = []
for _ in range(dims['nt'] + 1):
    q_exact.append(model_parameter['q_exact'])
q_exact = np.array(q_exact)

U = model.solve_state(q_exact)

Accordion(children=(HTML(value='', layout=Layout(height='16em', width='100%')),), titles=('Log Output',))

In [11]:
model.visualizer.visualize(U)

In [11]:
from pymor.discretizers.builtin.grids.rect import RectGrid


exact_assembled_fom, exact_assembled_fom_data = discretize_instationary_cg(exact_analytical_problem,
                                               diameter=dims['diameter'],
                                               preassemble= True,
                                               grid_type = RectGrid,
                                               nt = dims['nt'])

Accordion(children=(HTML(value='', layout=Layout(height='16em', width='100%')),), titles=('Log Output',))

In [12]:
np.max(np.abs(exact_assembled_fom.operator.matrix.todense() - model.A(model_parameter['q_exact']).matrix.todense()))

0.0005435370648187998

In [49]:
exact_assembled_fom.operator.matrix.todense() - model.A(model_parameter['q_exact']).matrix.todense()

matrix([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.10037288,  0.18153289, -0.00960643,  0.18153289,  0.4451158 ,
          0.04102501, -0.00960643,  0.04102501,  0.03011894],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , 

In [70]:
model.A(model_parameter['q_exact']).matrix.todense()

matrix([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ],
        [-0.30233497, -0.20896766, -0.30214886, -0.20896766,  3.1656319 ,
         -0.20821639, -0.30214886, -0.20821639, -0.30195934],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          1.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  1.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , 

In [71]:
exact_assembled_fom.operator.matrix.todense()

matrix([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ],
        [-0.20196209, -0.02743478, -0.31175529, -0.02743478,  3.6107477 ,
         -0.16719138, -0.31175529, -0.16719138, -0.27184039],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          1.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  1.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , 

In [12]:
U_ex = exact_assembled_fom.solve()

Accordion(children=(HTML(value='', layout=Layout(height='16em', width='100%')),), titles=('Log Output',))

In [13]:
model.visualizer.visualize(U_ex)

In [65]:
exact_assembled_fom.mass.matrix.todense()

matrix([[1.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        ],
        [0.        , 1.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 1.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 1.        , 0.        ,
         0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.11111111,
         0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         1.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 1.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 1.       

In [69]:
model.M.assemble().matrix.todense()

Accordion(children=(HTML(value='', layout=Layout(height='16em', width='100%')),), titles=('Log Output',))

matrix([[1.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        ],
        [0.        , 1.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 1.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 1.        , 0.        ,
         0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.11111111,
         0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         1.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 1.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 1.       